In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [ ]:
# Load the trained model
model = load_model('music_model.h5')

In [ ]:
# Preprocess the input image
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (32, 32))  # Resize to match the model's input size
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Pass the image through the model and get predictions
def predict_class(image_path):
    preprocessed_image = preprocess_image(image_path)
    predictions = model.predict(preprocessed_image)
    predicted_class_index = np.argmax(predictions, axis=1)
    return predicted_class_index[0]

# Main test dataset folder path
main_test_dataset_folder = "Test_dataset"

classes = ["सा", "रे", "ग", "म", "प", "ध", "नि", "रे॒", "ग॒", "ध॒", "नि॒", "म॑", 
           "सा\u0951", "रे\u0951", "ग\u0951", "म\u0951", "प\u0951", "ध\u0951", "रे॒\u0951", "ग॒\u0951", "म॑'", 
           "म\u093C", "म॑\u093C", "प\u093C", "ध॒\u093C", "ध\u093C", "नि॒\u093C", "नि\u093C", 
           "-", "४", "O", "X", "३", "२", "|"] 

# Initialize counters for correct and incorrect predictions
correct_predictions = 0
incorrect_predictions = 0
incorrect_files = []

# Predict classes for images in each class folder
for class_index, class_label in enumerate(os.listdir(main_test_dataset_folder), start=0):
    class_folder_path = os.path.join(main_test_dataset_folder, class_label)
    if os.path.isdir(class_folder_path):
        print(f"\nStarting predictions for class: {class_label}")
        # Predict classes for images in the current class folder
        for image_file in os.listdir(class_folder_path):
            image_path = os.path.join(class_folder_path, image_file)
            predicted_class_index = predict_class(image_path)
            predicted_class_label = classes[predicted_class_index]
            
            # Check if the predicted index matches the actual class index
            print(f'predicted: {predicted_class_index}, expected: {class_index}')
            if predicted_class_index == class_index:
                correct_predictions += 1
            else:
                incorrect_predictions += 1
                incorrect_files.append((image_file, predicted_class_label))
        
            print(f'File: {image_file}, Predicted class: {predicted_class_label}')

# Print summary
print("\nSummary:")
total_images = correct_predictions + incorrect_predictions
print(f"Total images: {total_images}")
print(f"Correct predictions: {correct_predictions}")
print(f"Incorrect predictions: {incorrect_predictions}")
if incorrect_files:
    print("\nIncorrect files:")
    for file, predicted_class in incorrect_files:
        print(f"File: {file}, Predicted class: {predicted_class}")